# Etapa 3 do Projeto Final

## Análise Exploratória e Consultas SQL

### 1. Setup e Carregamento do Dataset Limpo

Configuração do ambiente e carregamento do dataset final da Etapa 2.

In [3]:
# Imports
import pandas as pd
import duckdb 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Configs
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

# Conexão DB (DuckDB/PostgreSQL)
conn = duckdb.connect(database=':memory:', read_only=False)

# Carregar dataset limpo (SUBSTITUIR PELO CÓDIGO REAL DE CARREGAMENTO)
# Exemplo: df = conn.execute("SELECT * FROM airbnb_limpo").fetchdf()

# DataFrame simulado para demo (REMOVER)
# DataFrame simulado para demo (REMOVER)
data = {
    'ID': range(1, 11),
    'Bairro': ['Copacabana', 'Ipanema', 'Copacabana', 'Barra', 'Ipanema', 'Copacabana', 'Barra', 'Tijuca', 'Tijuca', 'Centro'],
    'Preco': [150.0, 300.0, 180.0, 120.0, 350.0, 160.0, 110.0, 80.0, 90.0, 100.0],
    'Tipo_Propriedade': ['Apartamento', 'Casa', 'Apartamento', 'Apartamento', 'Casa', 'Apartamento', 'Apartamento', 'Apartamento', 'Casa', 'Apartamento'],
    'Qtde_Avaliacoes': [10, 50, 5, 20, 100, 1, 0, 30, 2, 45],
    'Superhost': [True, True, False, False, True, False, False, True, False, True],
    'Avaliacao_Geral': [4.8, 4.9, 4.5, 4.2, 5.0, 4.0, 3.5, 4.7, 4.1, 4.6],
    'Data_Ultima_Avaliacao': pd.to_datetime(['2024-10-01', '2024-10-15', '2024-09-20', '2024-08-01', '2024-11-01', '2024-10-25', '2024-07-01', '2024-09-10', '2024-06-01', '2024-10-10'])
}
df = pd.DataFrame(data)
print(f"Dataset carregado (simulado) com {df.shape[0]} linhas e {df.shape[1]} colunas.")

df.head()

Dataset carregado (simulado) com 10 linhas e 8 colunas.


,ID,Bairro,Preco,Tipo_Propriedade,Qtde_Avaliacoes,Superhost,Avaliacao_Geral,Data_Ultima_Avaliacao
0,1,Copacabana,150.0,Apartamento,10,True,4.8,2024-10-01
1,2,Ipanema,300.0,Casa,50,True,4.9,2024-10-15
2,3,Copacabana,180.0,Apartamento,5,False,4.5,2024-09-20
3,4,Barra,120.0,Apartamento,20,False,4.2,2024-08-01
4,5,Ipanema,350.0,Casa,100,True,5.0,2024-11-01


### 2. Pipeline para Formato Tidy e Geração do Parquet

Garantir a estrutura final do dataset e salvá-lo em formato Parquet para a Etapa 4 (ML).

In [4]:
## Preparação Final
# Garantir tipos numéricos
df['Preco'] = pd.to_numeric(df['Preco'], errors='coerce')

# Limpeza final (opcional)
df_final = df.dropna(subset=['Preco', 'Avaliacao_Geral']).copy()

print(f"\nDataFrame final (tidy) com {df_final.shape[0]} linhas para Parquet.")

## Geração do Parquet
parquet_filename = 'airbnb_listings_final.parquet'
df_final.to_parquet(parquet_filename, index=False)

print(f"Dataset final em formato Parquet salvo como: {parquet_filename}")


DataFrame final (tidy) com 10 linhas para Parquet.
Dataset final em formato Parquet salvo como: airbnb_listings_final.parquet


### 3. Consultas SQL Analíticas

Execução de consultas SQL complexas, incluindo Funções de Janela, no banco de dados.

In [ ]:
# Carregar DF final para o DuckDB
conn.execute("CREATE OR REPLACE TABLE airbnb_limpo AS SELECT * FROM df_final")

# Query 1: Preço Médio por Bairro (Top N)
query_1 = """
WITH media_geral AS (
    SELECT AVG(Preco) AS preco_medio_geral
    FROM airbnb_limpo
)
SELECT
    Bairro,
    ROUND(AVG(Preco), 2) AS preco_medio_bairro,
    COUNT(ID) AS total_listings,
    ROUND((AVG(Preco) - (SELECT preco_medio_geral FROM media_geral)), 2) AS diferenca_media_geral
FROM
    airbnb_limpo
WHERE
    Preco > 0
GROUP BY
    Bairro
HAVING
    COUNT(ID) >= 1
ORDER BY
    preco_medio_bairro DESC
LIMIT 5;
"""

print("Consulta 1: Top 5 Bairros por Preço Médio\n")
try:
    print(conn.execute(query_1).fetchdf())
except Exception as e:
    print(f"Erro na Query 1: {e}.")

SyntaxError: invalid non-printable character U+00A0 (2095524928.py, line 30)

In [ ]:
# Query 2: Ranking por Avaliação (Função de Janela)
query_2 = """
SELECT
    Tipo_Propriedade,
    ID AS Listing_ID,
    Avaliacao_Geral,
    Qtde_Avaliacoes,
    RANK() OVER (
        PARTITION BY Tipo_Propriedade
        ORDER BY Avaliacao_Geral DESC, Qtde_Avaliacoes DESC
    ) AS Rank_Avaliacao_Tipo
FROM
    airbnb_limpo
WHERE
    Avaliacao_Geral IS NOT NULL
    AND Qtde_Avaliacoes > 1
ORDER BY
    Tipo_Propriedade, Rank_Avaliacao_Tipo
LIMIT 10;
"""

print("\nConsulta 2: Ranking de Listings por Tipo de Propriedade\n")
try:
    print(conn.execute(query_2).fetchdf())
except Exception as e:
    print(f"Erro na Query 2: {e}.")

In [ ]:
# Query 3: Distribuição Anual de Reviews
query_3 = """
SELECT
    STRFTIME(Data_Ultima_Avaliacao, '%Y') AS Ano_Review,
    COUNT(ID) AS Listings_com_Review_no_Ano,
    SUM(Qtde_Avaliacoes) AS Total_Reviews_Acumuladas
FROM
    airbnb_limpo
WHERE
    Data_Ultima_Avaliacao IS NOT NULL
GROUP BY
    Ano_Review
ORDER BY
    Ano_Review DESC;
"""

print("\nConsulta 3: Distribuição Anual de Reviews\n")
try:
    print(conn.execute(query_3).fetchdf())
except Exception as e:
    print(f"Erro na Query 3: {e}.")

# Fechar conexão
conn.close()

### 4. Análise Exploratória de Dados (EDA)

Obtenção de *insights* estatísticos e visuais a partir do `df_final`.

#### 4.1 Análise Univariada
Distribuição das variáveis principais.

In [ ]:
# Preço: Histograma
plt.figure(figsize=(10, 5))
sns.histplot(df_final['Preco'], bins=50, kde=True)
plt.title('Distribuição de Preços')
plt.xlabel('Preço')
plt.xlim(0, df_final['Preco'].quantile(0.95))
plt.show()

print("\nEstatísticas Descritivas do Preço:")
print(df_final['Preco'].describe())

# Tipo de Propriedade: Barplot
plt.figure(figsize=(12, 6))
top_types = df_final['Tipo_Propriedade'].value_counts().nlargest(10)
sns.barplot(x=top_types.index, y=top_types.values, palette="viridis")
plt.title('Top 10 Tipos de Propriedade')
plt.xlabel('Tipo de Propriedade')
plt.ylabel('Contagem')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#### 4.2 Análise Bivariada e Visualizações
Relação entre variáveis.

In [ ]:
# Preço vs. Superhost: Boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x='Superhost', y='Preco', data=df_final)
plt.title('Preço por Status de Superhost')
plt.xlabel('Superhost (True/False)')
plt.ylabel('Preço')
plt.ylim(0, df_final['Preco'].quantile(0.90))
plt.show()

# Avaliação Geral vs. Qtde_Avaliacoes: Scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Qtde_Avaliacoes', y='Avaliacao_Geral', data=df_final, alpha=0.6)
plt.title('Avaliação Geral vs. Qtde Avaliações')
plt.xlabel('Quantidade de Avaliações')
plt.ylabel('Avaliação Geral')
plt.xlim(0, df_final['Qtde_Avaliacoes'].quantile(0.99))
plt.show()

# Preço Médio por Bairro: Barplot
df_bairro = df_final.groupby('Bairro')['Preco'].agg(['mean', 'count']).reset_index()
df_bairro_top = df_bairro[df_bairro['count'] > 1].sort_values(by='mean', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x='Bairro', y='mean', data=df_bairro_top, palette='magma')
plt.title('Preço Médio por Bairro (Top N)')
plt.xlabel('Bairro')
plt.ylabel('Preço Médio')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### 5. Teste de Hipóteses

Teste de Hipótese (Exemplo): O preço médio de Superhosts é estatisticamente diferente do preço médio de não-Superhosts?

In [ ]:
# Filtrar dados
df_teste = df_final.dropna(subset=['Preco', 'Superhost'])

# Separar amostras
preco_superhost = df_teste[df_teste['Superhost'] == True]['Preco']
preco_nao_superhost = df_teste[df_teste['Superhost'] == False]['Preco']

# Limitar outliers (95º percentil)
if not preco_superhost.empty and not preco_nao_superhost.empty:
    limit = df_teste['Preco'].quantile(0.95)
    preco_superhost = preco_superhost[preco_superhost <= limit]
    preco_nao_superhost = preco_nao_superhost[preco_nao_superhost <= limit]

    print(f"Superhost Média={preco_superhost.mean():.2f}")
    print(f"Não-Superhost Média={preco_nao_superhost.mean():.2f}")

    # Teste t de Student (equal_var=False)
    if len(preco_superhost) > 1 and len(preco_nao_superhost) > 1:
        t_stat, p_valor = stats.ttest_ind(preco_superhost, preco_nao_superhost, equal_var=False)
        alpha = 0.05

        print(f"\nResultado Teste t: t={t_stat:.4f}, p={p_valor:.4f}")

        # Conclusão
        if p_valor < alpha:
            print(f"\nCONCLUSÃO: Rejeitamos H₀. Preço médio é estatisticamente diferente.")
        else:
            print(f"\nCONCLUSÃO: Não rejeitamos H₀. Preço médio não é estatisticamente diferente.")
    else:
        print("\nAVISO: Amostras insuficientes para Teste t.")
else:
    print("\nAVISO: Amostras de Superhost/Não-Superhost estão vazias.")